In [ ]:
# Import required libraries
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import sklearn

# Import necessary modules
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

# Keras specific
import keras
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.utils import to_categorical



In [ ]:
# !zip -r /content/deep-learning_data.zip /content


In [ ]:
df_full = pd.read_csv('./gym_monhtly_billing_training_set.csv') 
print(df_full.shape)
df_full.describe()
df_full.head()
df = df_full.drop("MEMBERSHIP NUMBER",axis=1,inplace=False)
df.head()

In [ ]:
# The first line of code creates an object of the target variable, while the second line of code gives the list of all the features after excluding the target variable, 'diabetes'.

# The third line does normalization of the predictors via scaling between 0 and 1. This is needed to eliminate the influence of the predictor's units and magnitude on the modelling process.

# The fourth line displays the summary of the normalized data. The target variable remains unchanged.

target_column = ['Paid'] 
predictors = list(set(list(df.columns))-set(target_column))
df[predictors] = df[predictors]/df[predictors].max()
df.describe()

In [ ]:
# Creating the Training and Test Datasets

X = df[predictors].values
y = df[target_column].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=40)
print(X_train.shape); print(X_test.shape)

In [ ]:
# one hot encode target colomn
# y_train = to_categorical(y_train)
# y_test = to_categorical(y_test)

count_classes = y_test.shape[1]
print(count_classes)

In [ ]:
#deep learning model

# model = Sequential()
# model.add(Dense(500, activation='relu', input_dim=8))
# model.add(Dense(100, activation='relu'))
# model.add(Dense(50, activation='relu'))
# model.add(Dense(1, activation='softmax'))

model = Sequential()
model.add(Dense(50, activation='relu', input_dim=8))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', 
              loss='binary_crossentropy', #binary
              metrics=['accuracy'])


In [ ]:
# build the model
model.fit(X_train, y_train, epochs=50)

In [ ]:
# Predict on the Test Data and Compute Evaluation Metrics;
pred_train= model.predict(X_train)
scores = model.evaluate(X_train, y_train, verbose=0)
print('Accuracy on training data: {}% \n Error on training data: {}%'.format(round(scores[1]*100,2), round((1 - scores[1])*100,2)))
 
pred_test= model.predict(X_test)
scores2 = model.evaluate(X_test, y_test, verbose=0)
print('Accuracy on test data: {}% \n Error on test data: {}%'.format(round(scores2[1]*100,2), round((1 - scores2[1])*100,2)))




In [ ]:
df_Forecastog = pd.read_csv('./gym_monthly_billing_forecast_set.csv') 
print(df_Forecastog.shape)
df_Forecastog.describe()
df_Forecastog.head()
drop_f=['MEMBERSHIP NUMBER','Total Dues ExVat']
df_Forecast = df_Forecastog.drop(drop_f,axis=1,inplace=False)
df_Forecast.head()

In [ ]:
#Predict values
predictors_Forecast = list(set(list(df_Forecast.columns)))
df_Forecast[predictors_Forecast] = df_Forecast[predictors_Forecast]/df_Forecast[predictors_Forecast].max()
df_Forecast.describe()

# Values to dataframe
N = df_Forecast[predictors_Forecast].values
pred= model.predict(N)
df = pd.DataFrame(pred)

# Sort Final Data
df_Forecastog["DL Estimator"] = round(df,2)
df_Forecastog['DL Outcome'] = df_Forecastog["DL Estimator"].map(lambda x: 1 if x >= .5 else 0)
df_Forecastog['Probabilty Adjusted Amount Low'] = df_Forecastog.apply(lambda row: row['Recurring Dues ExVat'] * row['DL Outcome'] ,axis=1)
df_Forecastog['Probabilty Adjusted Amount High'] = df_Forecastog.apply(lambda row: row['Total Dues ExVat'] * row['DL Outcome'] ,axis=1)


df_Forecastog.head(50)

In [ ]:
f1 = round(sum(df_Forecastog['Probabilty Adjusted Amount Low']))
f1Clean = "{:,}".format(f1)
f2 = round(sum(df_Forecastog['Probabilty Adjusted Amount High']))
f2Clean = "{:,}".format(f2)
f3 = f1-f2
f3Clean = "{:,}".format(f3)

print("Forecasted Collection Amount:",f1Clean,"AED","->",f2Clean,"AED",)
print()
print("Variance:",f3Clean,"AED")
print()
print('Accuracy on test data: {}% '.format(round(100*scores2[1], 2)))

